In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime

# Carpeta con formato DD-MM-YYYY
fecha_carpeta = datetime.now().strftime("%d-%m-%Y")
save_dir = os.path.join(os.getcwd(), f"SBS-RETASAS-{fecha_carpeta}")
os.makedirs(save_dir, exist_ok=True)

chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument("--headless")  # Opcional
driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 20)

base_url = "https://www.sbs.gob.pe/app/retasas/paginas/retasasInicio.aspx?p=D"
driver.get(base_url)

def get_options_text(id_select):
    select = Select(wait.until(EC.presence_of_element_located((By.ID, id_select))))
    opciones = [o.text.strip() for o in select.options if o.get_attribute("value").strip() != ""]
    print(f"Opciones de {id_select}: {opciones}")
    return opciones

def select_option(id_select, texto):
    select = Select(wait.until(EC.element_to_be_clickable((By.ID, id_select))))
    opciones = [o.text.strip() for o in select.options if o.get_attribute("value").strip() != ""]
    coincidencias = [opcion for opcion in opciones if texto.lower() in opcion.lower()]
    if not coincidencias:
        print(f"Opción '{texto}' no encontrada en {id_select}. Opciones disponibles: {opciones}")
        raise Exception(f"Opción '{texto}' no está en el select {id_select}")
    print(f"Seleccionando {coincidencias[0]} en {id_select}")
    select.select_by_visible_text(coincidencias[0])
    time.sleep(2)

def extraer_tabla_manual(driver):
    print("Extrayendo tabla...")
    table = wait.until(EC.presence_of_element_located((By.ID, "myTable")))
    thead = table.find_element(By.TAG_NAME, "thead")
    encabezados = [th.text.strip() for th in thead.find_elements(By.TAG_NAME, "th")]
    tbody = table.find_element(By.TAG_NAME, "tbody")
    filas = tbody.find_elements(By.TAG_NAME, "tr")
    datos = []
    for fila in filas:
        celdas = fila.find_elements(By.TAG_NAME, "td")
        fila_texto = [celda.text.strip().replace('%','').strip() for celda in celdas]
        datos.append(fila_texto)
    df = pd.DataFrame(datos, columns=encabezados)
    print(f"Tabla extraída con {len(df)} filas y {len(df.columns)} columnas")
    return df

def ajustar_anchos(worksheet, df):
    worksheet.set_column(0, 0, 20)
    worksheet.set_column(1, 1, 50)
    for i, col in enumerate(df.columns):
        max_len = max(df[col].astype(str).map(len).max(), len(col)) + 2
        worksheet.set_column(i, i, max_len)

# Inicio del recorrido
regiones = get_options_text("ddlDepartamento")

for region in regiones:
    select_option("ddlDepartamento", region)
    tipos_operacion = get_options_text("ddlTipoProducto")

    for tipo in tipos_operacion:
        select_option("ddlTipoProducto", tipo)
        productos = get_options_text("ddlProducto")

        for producto in productos:
            select_option("ddlProducto", producto)

            # Esperar a que ddlCondicion se actualice
            WebDriverWait(driver, 10).until(
                lambda d: len(get_options_text("ddlCondicion")) > 0
            )

            condiciones = get_options_text("ddlCondicion")

            if not condiciones:
                print(f"No hay condiciones para: {region} - {tipo} - {producto}, se omite.")
                continue

            for condicion in condiciones:
                try:
                    select_option("ddlCondicion", condicion)
                except Exception as e:
                    print(f"Salteando condición no encontrada: {condicion} | {e}")
                    continue

                btn_consultar = wait.until(EC.element_to_be_clickable((By.ID, "btnConsultar")))
                btn_consultar.click()
                print(f"Consultando: {region} - {tipo} - {producto} - {condicion}")
                time.sleep(4)

                wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, "ifrmContendedor")))

                try:
                    df = extraer_tabla_manual(driver)
                except Exception as e:
                    print(f"Error extrayendo tabla: {e}")
                    driver.switch_to.default_content()
                    driver.get(base_url)
                    # Re-seleccionar opciones después de recargar
                    wait.until(EC.presence_of_element_located((By.ID, "ddlDepartamento")))
                    wait.until(EC.presence_of_element_located((By.ID, "ddlTipoProducto")))
                    wait.until(EC.presence_of_element_located((By.ID, "ddlProducto")))
                    # Volver a seleccionar la región, tipo y producto actual
                    select_option("ddlDepartamento", region)
                    select_option("ddlTipoProducto", tipo)
                    select_option("ddlProducto", producto)
                    continue

                driver.switch_to.default_content()

                if df.empty:
                    print(f"Tabla vacía para {region}, {tipo}, {producto}, {condicion}, se omite guardar.")
                    driver.get(base_url)
                    # Re-seleccionar opciones después de recargar
                    wait.until(EC.presence_of_element_located((By.ID, "ddlDepartamento")))
                    wait.until(EC.presence_of_element_located((By.ID, "ddlTipoProducto")))
                    wait.until(EC.presence_of_element_located((By.ID, "ddlProducto")))
                    # Volver a seleccionar la región, tipo y producto actual
                    select_option("ddlDepartamento", region)
                    select_option("ddlTipoProducto", tipo)
                    select_option("ddlProducto", producto)
                    continue

                fecha_str = datetime.now().strftime("%d-%m-%Y")
                nombre_archivo = f"{fecha_str}-{region}-{tipo}-{producto}-{condicion}"
                for ch in [' ', '$', '.', ',', '|', '/']:
                    nombre_archivo = nombre_archivo.replace(ch, "-")
                if not nombre_archivo.lower().endswith(".xlsx"):
                    nombre_archivo += ".xlsx"

                ruta_guardado = os.path.join(save_dir, nombre_archivo)

                try:
                    with pd.ExcelWriter(ruta_guardado, engine='xlsxwriter') as writer:
                        workbook  = writer.book
                        worksheet = workbook.add_worksheet('Datos')
                        writer.sheets['Datos'] = worksheet

                        encabezados_personalizados = [
                            ['DEPARTAMENTO:', region],
                            ['TIPO DE PRODUCTO:', tipo],
                            ['PRODUCTO:', producto],
                            ['CONDICION:', condicion],
                            ['FECHA:', fecha_str]
                        ]
                        for i, row in enumerate(encabezados_personalizados):
                            worksheet.write(i, 0, row[0])
                            worksheet.write(i, 1, row[1])

                        df.to_excel(writer, sheet_name='Datos', startrow=6, index=False)
                        ajustar_anchos(worksheet, df)

                    print(f"Guardado archivo con encabezado y anchos ajustados: {ruta_guardado}")
                except Exception as e:
                    print(f"Error guardando Excel: {e}")

                driver.get(base_url)
                # Re-seleccionar opciones después de recargar
                wait.until(EC.presence_of_element_located((By.ID, "ddlDepartamento")))
                wait.until(EC.presence_of_element_located((By.ID, "ddlTipoProducto")))
                wait.until(EC.presence_of_element_located((By.ID, "ddlProducto")))
                # Volver a seleccionar la región, tipo y producto actual
                select_option("ddlDepartamento", region)
                select_option("ddlTipoProducto", tipo)
                select_option("ddlProducto", producto)

driver.quit()




   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Opciones de ddlDepartamento: ['AMAZONAS', 'ANCASH', 'APURIMAC', 'AREQUIPA', 'AYACUCHO', 'CAJAMARCA', 'CALLAO', 'CUSCO', 'HUANCAVELICA', 'HUANUCO', 'ICA', 'JUNIN', 'LA LIBERTAD', 'LAMBAYEQUE', 'LIMA', 'LORETO', 'MADRE DE DIOS', 'MOQUEGUA', 'PASCO', 'PIURA', 'PUNO', 'SAN MARTIN', 'TACNA', 'TUMBES', 'UCAYALI']\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 7 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 5 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 5 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 9 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 17 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 26 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 26 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 25 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 26 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 25 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 28 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 27 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 28 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 28 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 9 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 10 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 9 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 9 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 17 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 17 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 17 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 17 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 6 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 6 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 17 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 6 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 6 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 25 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 26 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 25 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 27 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 27 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 27 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 27 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 25 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 25 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 25 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 26 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 26 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 26 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 26 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 25 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 25 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 25 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 25 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 27 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 29 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 29 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 29 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 30 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 31 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 30 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 32 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 32 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 32 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 32 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 7 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 7 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 7 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 10 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 17 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 9 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 10 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 17 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 10 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 10 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 10 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 9 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 9 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 17 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 17 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 17 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 10 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 10 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n"
     ]
    }
   ],
   


   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Opciones de ddlDepartamento: ['AMAZONAS', 'ANCASH', 'APURIMAC', 'AREQUIPA', 'AYACUCHO', 'CAJAMARCA', 'CALLAO', 'CUSCO', 'HUANCAVELICA', 'HUANUCO', 'ICA', 'JUNIN', 'LA LIBERTAD', 'LAMBAYEQUE', 'LIMA', 'LORETO', 'MADRE DE DIOS', 'MOQUEGUA', 'PASCO', 'PIURA', 'PUNO', 'SAN MARTIN', 'TACNA', 'TUMBES', 'UCAYALI']\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 7 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 5 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 5 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AMAZONAS - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AMAZONAS-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando AMAZONAS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: ANCASH - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ANCASH-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando ANCASH en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 9 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: APURIMAC - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-APURIMAC-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando APURIMAC en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 17 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 26 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 26 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 25 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 26 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 25 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 28 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 27 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 28 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AREQUIPA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 28 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AREQUIPA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando AREQUIPA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 9 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 10 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 9 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 9 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 17 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 17 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 17 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: AYACUCHO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 17 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-AYACUCHO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando AYACUCHO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CAJAMARCA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CAJAMARCA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando CAJAMARCA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CALLAO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CALLAO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando CALLAO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: CUSCO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-CUSCO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando CUSCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 6 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 6 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANCAVELICA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANCAVELICA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 17 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: HUANUCO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-HUANUCO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANUCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 6 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 6 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: ICA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-ICA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando HUANCAVELICA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 25 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 26 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 25 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 27 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 27 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 27 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: JUNIN - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 27 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-JUNIN-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando JUNIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 25 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 25 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 25 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 26 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 26 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 26 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LA LIBERTAD - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 26 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LA-LIBERTAD-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando LA LIBERTAD en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 25 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 25 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 25 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LAMBAYEQUE - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 25 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LAMBAYEQUE-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando LAMBAYEQUE en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 27 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 29 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 29 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 29 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 30 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 31 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 30 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 32 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 32 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 32 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LIMA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 32 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LIMA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando LIMA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: LORETO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-LORETO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando LORETO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 7 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 7 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 7 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: MADRE DE DIOS - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MADRE-DE-DIOS-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando MADRE DE DIOS en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 10 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: MOQUEGUA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 17 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-MOQUEGUA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando MOQUEGUA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 9 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 10 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PASCO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PASCO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando PASCO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 23 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 22 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PIURA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 24 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PIURA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando PIURA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 13 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 17 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 10 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 10 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: PUNO - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-PUNO-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando PUNO en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 8 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: SAN MARTIN - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-SAN-MARTIN-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando SAN MARTIN en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: TACNA - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TACNA-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando TACNA en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 10 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 11 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 9 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 9 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 16 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 15 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 17 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 17 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: TUMBES - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 17 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-TUMBES-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando TUMBES en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Opciones de ddlTipoProducto: ['DEPOSITOS']\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Opciones de ddlProducto: ['AHORRO PERSONAS NATURALES', 'CTS', 'PLAZO FIJO EN DOLARES', 'PLAZO FIJO EN SOLES']\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Opciones de ddlCondicion: ['CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO', 'CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO', 'RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO']\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 12 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN DOLARES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 18 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-DOLARES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 14 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - AHORRO PERSONAS NATURALES - CUENTA DE AHORRO EN NUEVOS SOLES SIN COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-AHORRO-PERSONAS-NATURALES-CUENTA-DE-AHORRO-EN-NUEVOS-SOLES-SIN-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN DOLARES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 10 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-DOLARES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - AHORRO PERSONAS NATURALES - RENDIMIENTO DE CUENTAS DE AHORRO EN SOLES CON COBRO DE MANTENIMIENTO\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 10 filas y 4 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-AHORRO-PERSONAS-NATURALES-RENDIMIENTO-DE-CUENTAS-DE-AHORRO-EN-SOLES-CON-COBRO-DE-MANTENIMIENTO.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando AHORRO PERSONAS NATURALES en ddlProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Opciones de ddlCondicion: ['RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS', 'RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS']\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - CTS - RENDIMIENTO ASOCIADO A LOS DEPOSITOS CTS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 3 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-CTS-RENDIMIENTO-ASOCIADO-A-LOS-DEPOSITOS-CTS.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando CTS en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 1 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---1-500-A-360-DIAS.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 20 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - PLAZO FIJO EN DOLARES - DEPOSITOS A PLAZO POR US$. 3 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 19 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-PLAZO-FIJO-EN-DOLARES-DEPOSITOS-A-PLAZO-POR-US---3-000-A-360-DIAS.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN DOLARES en ddlProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Opciones de ddlCondicion: ['DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS', 'DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS']\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 5 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---5-000-A-360-DIAS.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 10 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---10-000-A-360-DIAS.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 2 500 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---2-500-A-360-DIAS.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n",
      "Seleccionando DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS en ddlCondicion\n",
      "Consultando: UCAYALI - DEPOSITOS - PLAZO FIJO EN SOLES - DEPOSITOS A PLAZO POR S/. 20 000 A 360 DIAS\n",
      "Extrayendo tabla...\n",
      "Tabla extraída con 21 filas y 2 columnas\n",
      "Guardado archivo con encabezado y anchos ajustados: C:\\Users\\Lenovo\\Downloads\\6to ciclo\\Analitica de datos\\Parcial Web Scrapping\\SBS-RETASAS-16-05-2025\\16-05-2025-UCAYALI-DEPOSITOS-PLAZO-FIJO-EN-SOLES-DEPOSITOS-A-PLAZO-POR-S---20-000-A-360-DIAS.xlsx\n",
      "Seleccionando UCAYALI en ddlDepartamento\n",
      "Seleccionando DEPOSITOS en ddlTipoProducto\n",
      "Seleccionando PLAZO FIJO EN SOLES en ddlProducto\n"
     ]
    }
   ],